In [75]:
import math
import string


def get_matrix(area: str):
    return list(map(lambda line: list(line), area.split("\n")))


def render_matrix(matrix):
    return "\n".join(["".join(line) for line in matrix])


# def get_antennas_in_range(matrix, x, y):
#     antennas = []
#     for angle in range(0, 360, 1):
#         for i in range(1, len(matrix) * 2):
#             lx = round(x + (i * math.sin(math.radians(angle))))
#             ly = round(y - (i * math.cos(math.radians(angle))))

#             if lx > -1 and lx < len(matrix[0]) and ly > -1 and ly < len(matrix):
#                 if matrix[y][x] == matrix[ly][lx]:
#                     if (lx, ly) not in antennas:
#                         antennas.append((lx, ly))
#     return antennas

def get_antennas_of_same_frequency(matrix, x, y):
    current = matrix[y][x]
    antennas = []
    for my in range(len(matrix)):
        for mx in range(len(matrix[y])):
            if (x != mx or y != my) and matrix[my][mx] == current :
                antennas.append((mx, my))
    return antennas


def get_distance(a: tuple, b: tuple):
    ax, ay = a
    bx, by = b

    x = abs(ax - bx)
    y = abs(ay - by)

    return math.sqrt(pow(x, 2) + pow(y, 2))


def get_placement(a: tuple, b: tuple):
    ax, ay = a
    bx, by = b

    x = ax - bx
    y = ay - by
    # print(f"{x=} {y=}")
    c = get_distance(a, b)

    nx = (2 * abs(x) * c) / c
    ny = (2 * abs(y) * c) / c

    px = int(ax + nx)
    py = int(ay + ny)

    if x > 0:
        px = int(ax - nx)

    if y > 0:
        py = int(ay - ny)

    return (px, py)


def get_antennas(matrix):
    antennas = []
    for y in range(len(matrix)):
        for x in range(len(matrix[y])):
            if matrix[y][x] not in string.punctuation:
                antennas.append((x, y))
    return antennas


def is_valid_point(matrix, x: int, y: int):
    if y > -1 and y < len(matrix):
        if x > -1 and x < len(matrix[y]):
            return True
    return False


# area = """
# ........................
# .......#................
# ........................
# .........O...#..........
# ............O...........
# ...........O............
# ........................
# ..........O.O...........
# ........................
# .........#...#..........
# ........................
# ........................
# """.strip()

# with open("test.txt", "r") as f:
#     test_area = f.read()


with open("input.txt", "r") as f:
    area = f.read().strip()


def visualise(area, antinodes, char="#"):
    matrix = get_matrix(area)
    for x, y in set(antinodes):
        matrix[y][x] = "\033[1;43m" + matrix[y][x] + "\033[0m"
    print(render_matrix(matrix))


def get_antinodes(area: str):
    matrix = get_matrix(area)
    antinodes = []

    print(f"Antenna count: {len(get_antennas(matrix))}")
    
    for x, y in get_antennas(matrix):
        for other_antenna in get_antennas_of_same_frequency(matrix, x, y):
            px, py = get_placement((x, y), other_antenna)
            if is_valid_point(matrix, px, py):
                antinodes.append((px, py))

    return tuple(set(antinodes))


antinodes = get_antinodes(area)

print(f"Antinodes count: {len(antinodes)}")

# visualise(area, antinodes)


Antenna count: 231
Antinodes count: 371


Correct: `371`

In [79]:
import json
with open("input.txt", "r") as f:
    area = f.read().strip()


# area = """
# T....#....
# ...T......
# .T....#...
# .........#
# ..#.......
# ..........
# ...#......
# ..........
# .x..#.....
# ..........
# """.strip()


def get_antennas_of_same_frequency(matrix, x, y):
    current = matrix[y][x]
    antennas = []
    for my in range(len(matrix)):
        for mx in range(len(matrix[y])):
            if (x != mx or y != my) and matrix[my][mx] == current :
                antennas.append((mx, my))
    return antennas

def get_resonant_locations(matrix: list, source_antenna: tuple, remote_antenna: tuple):
    nodes = [source_antenna, remote_antenna]
    source_node = tuple([*source_antenna])
    next_node = tuple([*remote_antenna])
    while True:
        px, py = get_placement(source_node, next_node)

        if is_valid_point(matrix, px, py):
            nodes.append((px, py))
            source_node = tuple([ *next_node ])
            next_node = (px, py)
            continue
        break
    return list(set(nodes))

def find_lone_antennas(matrix):
    antenna = {}
    out = []
    for y in range(len(matrix)):
        for x in range(len(matrix[y])):
            if matrix[y][x] in string.punctuation:
                continue
            freq = matrix[y][x]
            if freq not in antenna.keys():
                antenna[freq] = []
            antenna[freq].append((x, y))
    print(json.dumps(antenna))
    return list(map(lambda freq: antenna[freq][0], filter(lambda freq: len(antenna[freq]) == 1, antenna.keys())))
                                 

def get_resonant_antinodes(area: str):
    matrix = get_matrix(area)
    antinodes = []

    for x, y in get_antennas(matrix):
        for remote_antenna in get_antennas_of_same_frequency(matrix, x, y):
            nodes = get_resonant_locations(matrix, (x, y), remote_antenna)
            antinodes.extend(nodes)
    return tuple(set(antinodes))


antinodes = get_resonant_antinodes(area)

# print(find_lone_antennas(get_matrix(area)))
visualise(area, antinodes)

len(antinodes)

.C...............w.......................M.E......
...............G........V.............Q....M......
u........k...........V.y..3........Q..........4.a.
..........c.9........k..................i..7..a...
..............y.......................o....a......
.......C...........6.......y.............E........
.............................5....x............i..
...............c.....wy..V.......5..............E.
........k.......c....G..I............o.........m..
............C....s......G......o..........5.......
......................Q...............5....e...4i.
.....I.....................................m.....j
....9K.T.....I...c......w...................X.....
................I.........w....f............3..e.N
C............9..........6..............7...3......
...Z........K.......T.................6...........
......Z..................6...............HN.E.m...
...K...........................1....N...e.o..X....
............hz......................7........j....
.........9......U.R......n.....

1230

# Incorrect


- *1100* - Without antennas
- **1223** - That's not the right answer; your answer is too low.
- **1227** - That's not the right answer; your answer is too low.
- **1229** - Correct!
- **1230** - That's not the right answer; your answer is too high.